In [73]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time

In [74]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [75]:
#holdings=['RHHBY','BIG','DOCU','CLX']RH, ARKF, JHG
holdings=['CDNS']

In [83]:
holdings='SPY,CMG,DOCU,OLLI,SHOP,MBUU,ADBE,CCL,BL,QQQ,MDB,BLK,CLF,CDNS,MELI,NOW,NYT,CWH,ECL,DDOG,BIG,TREX,TW,DKNG,ARKF,RH,BAK,JHG,CROX,THO,MAR'



In [84]:
holdings=holdings.split(',')

In [85]:
#ticker, #1wkret #1wkmean, 4wk, 4wkmean,k, kchange, kchangemean,k_change percentile, std, stdmean
res=[[],[],[],[],[],[],[],[],[],[],[]]

In [86]:
c=1
for ticker in holdings:
    #print(ticker)
    c+=1
    if c==5:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_WEEKLY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )
    print(ticker)#,response.json())
    spy=[[],[],[],[],[],[],[]]
    for key, val in response.json()['Weekly Adjusted Time Series'].items():
        spy[0].append(float(val['1. open']))
        spy[1].append(float(val['2. high']))
        spy[2].append(float(val['3. low']))
        spy[3].append(float(val['4. close']))
        spy[4].append(float(val['5. adjusted close']))
        #spy[5].append(float(val['6. volume']))
        #spy[6].append(float(val['7. dividend amount']))
    t=list(response.json()['Weekly Adjusted Time Series'].keys())
    time.sleep(1)
    '''
    try:
        if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
            #print('Notlive')
            d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'False','chartLast':'1'}
            response = requests.get(
            'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format(ticker)#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
            ,params=d,
            )
            
            spy[0].insert(0,float(response.json()[0]['open']))
            spy[1].insert(0,float(response.json()[0]['high']))
            spy[2].insert(0,float(response.json()[0]['low']))
            spy[3].insert(0,float(response.json()[0]['close']))
            spy[4].insert(0,float(response.json()[0]['close']))
            t.insert(0,str(response.json()[0]['date']))

            #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
    except:
        print(ticker,'hi')
    '''    
        
    spy=pd.DataFrame({'Date':t,
                 'open':spy[0],
                  'high':spy[1],
                  'low':spy[2],
                  'close':spy[3],
                  'adj_close':spy[4]#,
                #  'volume':spy[5],
                #  'dividend':spy[6],
                 })
    spy=spy.astype({'Date': 'datetime64'})
    spy=spy.sort_values(by='Date')   
    spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1
    spy['highest_high']=spy.high.rolling(12).max()
    spy['lowest_low']=spy.low.rolling(12).min()
    spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
    spy['k']=spy.k.rolling(3).mean()
    spy['k_change'] = spy.k/spy.k.shift(3)-1
    spy['rstd']=spy.adj_close.rolling(12).std()
    spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1
    d={'ticker':ticker,'k_change':spy.k_change.values[-1]}
    res[0].append(ticker)
    res[1].append(spy.ret1wk.values[-1])
    res[2].append(spy.ret1wk.rolling(3).mean().values[-1])
    res[3].append(spy.ret4wk.values[-1])
    res[4].append(spy.ret4wk.rolling(3).mean().values[-1])
    res[5].append(spy.k.values[-1])
    res[6].append(spy.k_change.values[-1])
    res[7].append(spy.k_change.rolling(3).mean().values[-1])
    res[8].append(get_percentile(spy.k_change))
    res[9].append(spy.rstd.values[-1])
    res[10].append(spy.rstd.rolling(3).mean().values[-1])
    
    
    
    

SPY
CMG
DOCU
OLLI
SHOP
MBUU
ADBE
CCL
BL
QQQ
MDB
BLK
CLF
CDNS
MELI
NOW
NYT
CWH
ECL
DDOG
BIG
TREX
TW
DKNG
ARKF
RH
BAK
JHG
CROX
THO
MAR


In [87]:
res=pd.DataFrame({'ticker':res[0], 
            'ret1wk':res[1],
             'ret1wk_mean':res[2],
             'ret4wk':res[3],
             'ret4wk_mean':res[4],
             'k':res[5],
             'k_change':res[6],     
             'k_change_mean':res[7],
             'k_change_percentile':res[8],
             'rstd':res[9],
             'rstd_mean':res[10]
             }
            )

In [88]:
# Amount of time above current K
res.sort_values(by='ret1wk_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
7,CCL,0.366582,0.195519,0.683099,0.276272,0.544917,2.249836,1.226191,96.59,3.068535,3.300532
25,RH,0.135184,0.170402,0.592252,0.463250,0.952000,1.115336,1.094815,89.74,50.396854,41.222629
12,CLF,0.241379,0.166076,0.580488,0.261031,0.809286,1.261236,1.023542,87.93,0.888005,0.697846
28,CROX,0.179407,0.121652,0.407917,0.254566,0.903402,0.743898,0.659171,84.27,6.136463,5.244685
17,CWH,0.043909,0.121281,0.394073,0.778616,0.895580,0.282129,0.968220,73.14,6.639495,5.760917
27,JHG,0.147959,0.120461,0.403061,0.244032,0.903574,1.098124,1.123891,87.40,3.464357,2.750804
29,THO,0.160093,0.119205,0.398015,0.273220,0.958819,0.382926,0.589237,73.22,20.636600,17.798144
23,DKNG,-0.009068,0.116873,0.345878,0.495220,0.902438,0.048645,0.110728,58.62,10.009112,8.362758
30,MAR,0.219887,0.111143,0.353561,0.147718,0.697253,0.854293,0.531854,85.30,11.356414,10.993976
26,BAK,0.070736,0.107694,0.357494,0.334303,0.800949,1.233844,0.945894,88.12,1.819859,1.646088


In [89]:
res.sort_values(by='ret4wk',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
7,CCL,0.366582,0.195519,0.683099,0.276272,0.544917,2.249836,1.226191,96.59,3.068535,3.300532
25,RH,0.135184,0.170402,0.592252,0.463250,0.952000,1.115336,1.094815,89.74,50.396854,41.222629
12,CLF,0.241379,0.166076,0.580488,0.261031,0.809286,1.261236,1.023542,87.93,0.888005,0.697846
28,CROX,0.179407,0.121652,0.407917,0.254566,0.903402,0.743898,0.659171,84.27,6.136463,5.244685
27,JHG,0.147959,0.120461,0.403061,0.244032,0.903574,1.098124,1.123891,87.40,3.464357,2.750804
29,THO,0.160093,0.119205,0.398015,0.273220,0.958819,0.382926,0.589237,73.22,20.636600,17.798144
17,CWH,0.043909,0.121281,0.394073,0.778616,0.895580,0.282129,0.968220,73.14,6.639495,5.760917
26,BAK,0.070736,0.107694,0.357494,0.334303,0.800949,1.233844,0.945894,88.12,1.819859,1.646088
30,MAR,0.219887,0.111143,0.353561,0.147718,0.697253,0.854293,0.531854,85.30,11.356414,10.993976
23,DKNG,-0.009068,0.116873,0.345878,0.495220,0.902438,0.048645,0.110728,58.62,10.009112,8.362758


In [90]:
res.sort_values(by='k_change_mean',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
7,CCL,0.366582,0.195519,0.683099,0.276272,0.544917,2.249836,1.226191,96.59,3.068535,3.300532
27,JHG,0.147959,0.120461,0.403061,0.244032,0.903574,1.098124,1.123891,87.40,3.464357,2.750804
25,RH,0.135184,0.170402,0.592252,0.463250,0.952000,1.115336,1.094815,89.74,50.396854,41.222629
12,CLF,0.241379,0.166076,0.580488,0.261031,0.809286,1.261236,1.023542,87.93,0.888005,0.697846
17,CWH,0.043909,0.121281,0.394073,0.778616,0.895580,0.282129,0.968220,73.14,6.639495,5.760917
26,BAK,0.070736,0.107694,0.357494,0.334303,0.800949,1.233844,0.945894,88.12,1.819859,1.646088
5,MBUU,0.137916,0.090073,0.288873,0.302528,0.897513,0.459034,0.835273,76.19,10.839577,9.345439
16,NYT,0.057354,0.040989,0.127787,0.134830,0.945529,0.436461,0.808997,74.49,4.260755,3.688913
28,CROX,0.179407,0.121652,0.407917,0.254566,0.903402,0.743898,0.659171,84.27,6.136463,5.244685
29,THO,0.160093,0.119205,0.398015,0.273220,0.958819,0.382926,0.589237,73.22,20.636600,17.798144


In [92]:
res.sort_values(by='ret1wk',ascending=False)

,ticker,ret1wk,ret1wk_mean,ret4wk,ret4wk_mean,k,k_change,k_change_mean,k_change_percentile,rstd,rstd_mean
7,CCL,0.366582,0.195519,0.683099,0.276272,0.544917,2.249836,1.226191,96.59,3.068535,3.300532
12,CLF,0.241379,0.166076,0.580488,0.261031,0.809286,1.261236,1.023542,87.93,0.888005,0.697846
30,MAR,0.219887,0.111143,0.353561,0.147718,0.697253,0.854293,0.531854,85.30,11.356414,10.993976
28,CROX,0.179407,0.121652,0.407917,0.254566,0.903402,0.743898,0.659171,84.27,6.136463,5.244685
29,THO,0.160093,0.119205,0.398015,0.273220,0.958819,0.382926,0.589237,73.22,20.636600,17.798144
27,JHG,0.147959,0.120461,0.403061,0.244032,0.903574,1.098124,1.123891,87.40,3.464357,2.750804
5,MBUU,0.137916,0.090073,0.288873,0.302528,0.897513,0.459034,0.835273,76.19,10.839577,9.345439
25,RH,0.135184,0.170402,0.592252,0.463250,0.952000,1.115336,1.094815,89.74,50.396854,41.222629
18,ECL,0.079876,0.056550,0.178742,0.102981,0.948476,0.153693,0.223145,67.28,25.923799,22.406072
26,BAK,0.070736,0.107694,0.357494,0.334303,0.800949,1.233844,0.945894,88.12,1.819859,1.646088


In [9]:
msft=spy.copy(deep=True)

In [14]:
msft['ret4wk']=msft.adj_close/msft.adj_close.shift(3)-1
spy['ret4wk']=spy.adj_close/spy.adj_close.shift(3)-1

In [16]:
spy['highest_high']=spy.high.rolling(12).max()
spy['lowest_low']=spy.low.rolling(12).min()
spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
spy['k']=spy.k.rolling(3).mean()
spy['k_change'] = spy.k/spy.k.shift(3)-1

msft['highest_high']=msft.high.rolling(12).max()
msft['lowest_low']=msft.low.rolling(12).min()
msft['k']=(msft.close-msft.lowest_low)/(msft.highest_high-msft.lowest_low)
msft['k']=msft.k.rolling(3).mean()
msft['k_change'] = msft.k/msft.k.shift(3)-1




In [22]:
spy['rstd']=spy.adj_close.rolling(12).std()
msft['rstd']=msft.adj_close.rolling(12).std()

In [25]:
msft['ret1wk']=msft.adj_close/msft.adj_close.shift(1)-1
spy['ret1wk']=spy.adj_close/spy.adj_close.shift(1)-1

In [26]:
msft.tail(10)

,Date,open,high,low,close,adj_close,volume,dividend,ret4wk,highest_high,lowest_low,k,k_change,rstd,ret1wk
9,2020-03-20,140.00,150.15,135.0000,137.35,137.35,421347734.0,0.0,-0.152213,190.70,135.00,0.226001,-0.641828,12.881111,-0.135239
8,2020-03-27,137.01,156.66,132.5200,149.70,149.70,358866845.0,0.0,-0.073467,190.70,132.52,0.242002,-0.424578,13.538858,0.089916
7,2020-04-03,152.44,164.78,150.0100,153.83,153.83,290191457.0,0.0,-0.031480,190.70,132.52,0.234586,-0.239293,13.876028,0.027589
6,2020-04-09,160.32,170.00,157.5800,165.14,165.14,229630744.0,0.0,0.202330,190.70,132.52,0.407414,0.802706,13.859286,0.073523
5,2020-04-17,164.35,180.00,162.3000,178.60,178.60,238965670.0,0.0,0.193053,190.70,132.52,0.572992,1.367711,14.477445,0.081507
4,2020-04-24,176.63,178.75,166.1102,174.55,174.55,194621072.0,0.0,0.134694,190.70,132.52,0.691704,1.948616,14.676007,-0.022676
3,2020-05-01,176.59,180.40,169.3900,174.57,174.57,212119968.0,0.0,0.057103,190.70,132.52,0.745732,0.830404,13.910374,0.000115
2,2020-05-08,174.49,185.00,173.8000,184.68,184.68,158579959.0,0.0,0.034043,188.18,132.52,0.794096,0.385877,13.888795,0.057914
1,2020-05-15,183.15,187.51,175.6800,183.16,183.16,196126640.0,0.0,0.049327,187.51,132.52,0.860257,0.243678,14.351698,-0.008230
0,2020-05-19,185.75,186.60,183.4900,183.63,183.63,61664938.0,0.0,0.051899,187.51,132.52,0.929152,0.245960,15.227043,0.002566


In [38]:
spy.ret4wk.rolling(3).mean().values[-1]
spy.k_change.rolling(3).mean().values[-1]
spy.rstd.rolling(3).mean().values[-1]

22.84021899844733

In [47]:
spy.ret1wk.values[-1]

0.019875646220483656